In [ ]:
using JuMP


In [43]:
import Pkg
Pkg.add("Ipopt")

   Resolving package versions...


   Installed ASL_jll ─── v0.1.3+0
   Installed Ipopt_jll ─ v300.1400.400+0
   Installed Ipopt ───── v1.1.0


    Updating `~/.julia/environments/v1.8/Project.toml`
  [b6b21f68] + Ipopt v1.1.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [b6b21f68] + Ipopt v1.1.0
  [ae81ac8f] + ASL_jll v0.1.3+0
⌃ [06985876] ↓ Clp_jll v100.1700.700+1 ⇒ v100.1700.601+0
⌅ [9cc047cb] + Ipopt_jll v300.1400.400+0
⌅ [d00139f3] ↓ METIS_jll v5.1.2+0 ⇒ v5.1.1+0
⌅ [d7ed1dd3] ↓ MUMPS_seq_jll v500.500.101+0 ⇒ v5.4.1+0
        Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ cannot be upgraded. To see why use `status --outdated -m`


Precompiling 

project...


  ✓ ASL_jll
  ✓ METIS_jll


  ✓ MUMPS_seq_jll


  ✓ Ipopt_jll
  ✓ Clp_jll


  ✓ Ipopt


  ✓ Clp
  7 dependencies successfully precompiled in 40 seconds. 55 already precompiled.
  4 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


### COM361 &mdash; Introdução a Otimização &mdash; 2022, Prof. Amit ###

# Otimização de Wifi #

#### Pedro Henrique de Jesus Teixeira (pedroteixeir@poli.ufrj.br), Diego Nunes Gonçalves Freitas (diegongfreitas@poli.ufrj.br)

*****

### Índice

1. [Introdução](#1.-Introdução)
1. [Modelo Matemático](#2.-Modelo Mathemático)
1. [Solução](#3.-Solução)
1. [Resultados e Discussão](#4.-Resultados-e-discussão)
  1. [Subseção Opcional](#4.A.-Acrescente-subseções-se-necessário)
1. [Conclusão](#5.-Conclusão)
1. [Referências bibliográficas] (#6.-Referências bibliográficas)

## 1. Introdução ##

O projeto a ser apresentado ao longo desse relatório consiste em um mecânismo de otimização do posicionamento de um roteador wifi em um estabelecimento ou residência. Faremos isso porcurando o melhor ponto de instalação de um roteador objetivando a maximização da força do sinal que chega aos dispositivos conectados. 

É muito comum perceber cômodos de sua casa em que o sinal wifi chega muito enfraquecido ou simplesmente não chega, chamaremos essas regiões de Zonas Mortas. Denominamos de Zonas Críticas os cômodos nos quais há um dispositivo conectado a rede. Além de maximizar o sinal em cada dispositivo conectado precisamos, também, eliminar a possibilidade de formação de Zonas Mortas em Zonas Críticas. A complexidade dos cenários de teste do nosso algoritmo cresce gradualmente, conforme aumentamos a planta do imóvel e adicionamos fontes de interferência como espelhos, madeira, móveis metálicos, etc. 

O primeiro padrão de conexão wifi surgiu em 1997. Com o passar dos anos essa tecnologia se popularizou e, nos dias atuais, quase todo estabelecimento ou residência possui uma rede de internet via Wifi. Com a expansão dessa tecnologia suas qualidades e defeitos foram ficando mais evidentes. Um dos mais evidentes defeitos é a grande dependência do funcionamento ótimo da rede ao posicionamento do roteador wifi. Se mal posicionado em uma residência, o roteador pode acabar entregando, aos seus dispositivos conectados, porcentagens incrivelmente pequenas do sinal original. 

Nosso projeto tratará sobre redes [WLAN](https://pt.wikipedia.org/wiki/Rede_de_área_local_sem_fio), ou seja, redes de area local sem fio que usam ondas de rádio para fazer a conexão entre os dispositivos. Para fins de simplificação, os dados relacionados à perda de sinal ao longo da distância e por meio de interferências foram sintetizados e previamente estabelecidos.

A partir desse ponto apresentaremos o desenvolvimento do projeto, começando pelo modelo matématico na seção 2. Em seguida, na seção 3, apresentamos nossa solução de otimização, desenvolvida em Julia 1.8.0, possível de ser rodada nesse notebook Jupyter. Na seção 4, evidenciamos nossos resultados para cada cenário, variando em graus de complexidade, através de plots simulando um ambiente de rede. Na seção 5, finalizamos a apresentação do projeto apresentando nossas conclusões acerca dos resultados obtidos e dos métodos utilizados, ressaltando aspectos passíveis de melhorias. Por fim, na seção 6, apresentamos as referencias biubliográficas utilizadas ao longo do desenvolvimento do projeto.

## 2. Modelo matemático ##
Para abordar melhor nosso problema representamos o ambiente do imóvel trabalhado como uma matriz $A_{mXn}$. Começaremos apresentando nossas variáveis
de decisão. 

Denominamos $C_{ij}$, um posição da matriz em que há um dispositivo conectado a rede, sendo $c$ o número de dispositivos.

Denominamos $R_{ij}$, uma posição da matriz em que há um roteador.

O valor registrado em cada posição da matriz determinará o que esta presente naquele local da forma:
$$
a_{ij} = 
\begin{cases}
2, \text{ caso haja um roteador naquele local}\\
1, \text{ caso haja um dispositivo conectato naquele local}\\
0, \text{ caso não haja nada relacionado a rede naquele local}
\end{cases}
$$

Sopomos então que, a distância $d$ entre um roteador e um dispositivo, dada por:
$$
d = R_{ij} - C_{ij}
$$
influencia linearmente na força $f_{ij}$ do sinal que chega ao dispositivo na posição ij, da seguinte forma.
$$
f_{ij} = 1 - 0.2d 
$$

Primeiramente a posição de cada dispositivo sera dada aleatoriamente na matriz. 

Nosso objetivo será encontrar a posição para o roteador que maximize a soma das forças do sinal em cada dispositivo. 

Além disso, evitaremos grandes variações para a força do sinal em cada dispositivo, limitando o desvio padrão entre eles, sendo esse denominado $\sigma_f$.

Exemplificando o posicionamento ótimo de um roteador para um caso simples:
$$
\begin{bmatrix}
  1 & 0 & 0 & 2 & 0 & 0 & 1\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

Dessa forma, nosso problema de otimização em forma padrão fica assim:
$$
\begin{aligned}
\underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& \sum_{k=0}^{c}{f_k(d)} \\
\text{sujeito a:}\qquad& \sigma_{f(d)} \le 0,4\\
& d > 0
\end{aligned}
$$

Vamos trabalhar ainda com casos em que há objetos no estabelecimento que causem interferência no sinal, gerando Zonas mortas. Para isso iremos alterar um pouco nosso cenário:

$$
a_{ij} = 
\begin{cases}
3, \text{ caso haja uma fonte de interferência naquele local}\\
2, \text{ caso haja um roteador naquele local}\\
1, \text{ caso haja um dispositivo conectato naquele local}\\
0, \text{ caso não haja nada relacionado a rede naquele local}
\end{cases}
$$

Exemplificando, agora, o posicionamento ótimo de um roteador para um caso com interferência:

Se mantivermos o roteador na mesma posição um dos dispositivos ficaria em uma zona morta
$$
\begin{bmatrix}
  1 & 0 & 0 & 2 & 3 & 0 & 1\\
  0 & 0 & 0 & 0 & 3 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$
Para solucionar esse problema o roteador deve ser movido de forma a alterar a região de Zona morta, da seguinte forma:
$$
\begin{bmatrix}
  1 & 0 & 0 & 0 & 3 & 0 & 1\\
  0 & 0 & 0 & 0 & 3 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 2 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

Isso encerra o modelo matemático do nosso projeto. Em seguida apresentaremos nossa solução.



## 3. Solução ##

Nesta seção, coloque seu código em Julia + JuMP e resolva o problema proposto. Seu código deve ser limpo (não macarrônico!), de fácil leitura, bem comentado e anotado e deve compilar sem erros em Julia 1.x, x $\geq 1$! Não valem códigos em outras linguagens. **Vou rodar seu código para avaliar seu projeto**. Sugiro a utilização de múltiplos blocos de códigos separados por blocos de texto (células Markdown) explicando as várias partes da sua solução. Sugiro também a resolução de várias versões do seu problema, com modelos e hipóteses diferentes.

É permitido chamar pacotes externos, mas evite a utilização de bibliotecas exóticas (pois, em geral, não rodam em todas as versões de Julia, e terei que instalar a mesma versão que você usou, ou rodar na plataforma Google Colab, que gostaria de evitar).

In [60]:
# Este nosso modelo matematico
using JuMP, Ipopt, Statistics
m = Model(Ipopt.Optimizer)
apartamento = zeros(4,4)                                     # esta é matriz que define o tamanho do apartamento 
interferencias = [ [[] , []] ]
dispositivos = [[0,0],[2,2]]                    # as posições dos dispositivos no nosso sistema de coordenadas        
@variable(m, 0 <= x <= size(apartamento)[1])                            # as posições devem ser positivas
@variable(m, 0 <= y <= size(apartamento)[2])                            # as posições devem ser positivas e os roteadores da matriz não podem ser posicionados fora dos limites do apartamento

function force(x,y)
    forca = []
    for i in dispositivos
        distancia = sqrt((x-i[1])^2+(y-i[2])^2) # a distancia entre o roteador e os dispositivos ´pode ser calculado pela distancia euclidiana
        f_= 1-0.2*distancia                     # a forca do sinal definida pela funcao apresentada no modelo matematico
        if f_ < 0 
            f_ = 0                               # não é possivel ter sinais negativos
        end
        push!(forca,f_)
    end 
    return sum(forca)
end
function standart_dev(x,y)
    forca = []
    for i in dispositivos
        distancia = sqrt((x-i[1])^2+(y-i[2])^2) # a distancia entre o roteador e os dispositivos ´pode ser calculado pela distancia euclidiana
        f_= 1-0.2*distancia                     # a forca do sinal definida pela funcao apresentada no modelo matematico
        if f_ < 0 
            f_ = 0                               # não é possivel ter sinais negativos
        end
        push!(forca,f_)
    end 
    return Statistics.std(forca)
end
@NLconstraint(m, standart_dev(x,y)<= 0.4)
@NLobjective(m, Max, force(x,y))                 # queremos maximizar o número de cavalos
optimize!(m)

println("x =", JuMP.value.(x))     # imprime o resultado na tela
println("y =", JuMP.value.(y))     # imprime o resultado na tela


┌ Warning: Function standart_dev automatically registered with 2 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :standart_dev, 2, standart_dev; autodiff = true)
│ ```
└ @ MathOptInterface.Nonlinear /home/pedro/.julia/packages/MathOptInterface/Ohzb2/src/Nonlinear/operators.jl:359
┌ Warning: Function force automatically registered with 2 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :force, 2, force; autodiff = true)
│ ```
└ @ MathOptInterface.Nonlinear /home/pedro/.julia/packages/MathOptInterface/Ohzb2/src/Nonlinear/operators.jl:359


This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.4343146e+00 0.00e+00 1.85e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00  

**Tenha certeza de que seu código compila corretamente! Rodarei seu código!**

## 4. Resultados e discussão ##

Neste seção, os resultados obtidos serão exibidos e discutidos. Mostre figuras, gráficos, imagens, curvas de compromisso, e o que mais puder melhor ilustrar seus resultados. A discussão deverá explicar o que significam os resultados e como interpretá-los. As limitações da sua abordagem/modelo também devem ser colocadas, bem como uma análise da sensibilidade dos resultados em relação às hipóteses feitas.


Utilize plots (veja exemplos  `PyPlot` [aqui](https://gist.github.com/gizmaa/7214002))

Aqui está um exemplo de uma tabela (em Markdown):

| Tabelas        | São           | Boas  |
| ------------- |:-------------:| -----:|
| col 3 é      | alinhado à direita |\$1600 |
| col 2 é      | centrado      |  \$12 |
| texto | também serve      |   \$1 |

### 4.A. Subseções devem ser utilizadas para organizar seu texto.

#### 4.A.a. ou até subsubseções.

## 5. Conclusão ##

Faça um resumo do que encontrou e dos seus resultados, e fale de pelo menos uma direção na qual  seu trabalho pode ser desenvolvido no futuro, algo que poderia ser interessante em decorrência do seu projeto.


## 6. Referências bibliográficas ##

[Optimization Models and Methods for Planning Wireless Mesh Networks](https://www.researchgate.net/publication/220447584_Optimization_Models_and_Methods_for_Planning_Wireless_Mesh_Networks)

[Mesh](https://blog.intelbras.com.br/o-que-e-rede-mesh-e-quais-suas-vantagens/)